In [1]:
import geemap

In [2]:
m =geemap.Map()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=K4P9T4RMnJ_EJduKj2Sig0gE4dVm1VzTUupByzzVtR8&tc=qncjjoq3DS5hnqvAuq7gvyfCMJwaLn4S-b9l9HvgmZw&cc=xowFnv111LtdnCToZSIazvuJqL_cnm_3aM0Y_62Qag0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkYglxkJzoDQLg1BRMh-KYgleIkIiZDRBiVGRlM-yXS0CrONWfiMkQ

Successfully saved authorization token.


In [3]:
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [8]:
# Install required libraries
!pip install folium
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 44.5 MB/s eta 0:00:00


In [9]:
from flask import Flask, jsonify, request
from PIL import Image
from io import BytesIO
import folium
import os
import requests
import rasterio
from rasterio.plot import show

app = Flask(__name__)

LANDSAT_API_URL = "https://earthexplorer.usgs.gov/download/12864/LC08_L1TP_016037_20190826_20190903_01_T1.tar.gz"

def download_landsat_image(output_path):
    response = requests.get(LANDSAT_API_URL)
    with open(output_path, 'wb') as f:
        f.write(response.content)

def calculate_ndvi(red_band, nir_band):
    ndvi = (nir_band - red_band) / (nir_band + red_band)
    return ndvi

@app.route('/api/process_image', methods=['POST'])
def api_process_image():
    data = request.get_json()
    image_path = data.get('image_path')

    # Download Landsat image (replace this with your own method)
    landsat_image_path = "/content/DJI_0302.jpg"
    download_landsat_image(landsat_image_path)

    # Read Landsat image with rasterio
    with rasterio.open(landsat_image_path) as src:
        red_band = src.read(4)  # Red band
        nir_band = src.read(5)  # NIR band

    # Calculate NDVI
    ndvi = calculate_ndvi(red_band, nir_band)

    # Determine water quality based on NDVI
    ndvi_mean = ndvi.mean()
    if ndvi_mean > 0.2:
        water_quality = "Water quality is good."
    elif 0.1 <= ndvi_mean <= 0.2:
        water_quality = "Water quality is moderate."
    else:
        water_quality = "Water quality is poor. Action may be needed."

    # Create a folium map
    map_center = [0, 0]  # Replace with the actual coordinates
    map_zoom = 5
    mymap = folium.Map(location=map_center, zoom_start=map_zoom)

    # Add a marker for the image location
    folium.Marker(location=map_center, popup="Image Location").add_to(mymap)

    # Save the map as an HTML file
    map_path = "/path/to/map.html"
    mymap.save(map_path)

    return {
        "image_path": image_path,
        "ndvi": ndvi_mean,
        "water_quality": water_quality,
        "map_path": map_path
    }

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [11]:
import ee
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import folium
from folium import plugins
from datetime import datetime as dt

def get_geotagging(exif):
    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging

def convert_to_degrees(value):
    d = value[0]
    m = value[1]
    s = value[2]
    return d + (m / 60.0) + (s / 3600.0)

def get_gps_coordinates(image_path):
    try:
        img = Image.open(image_path)
        exif_data = img._getexif()

        if exif_data is not None:
            geotags = get_geotagging(exif_data)
            if geotags:
                latitude = geotags.get('GPSLatitude')
                longitude = geotags.get('GPSLongitude')

                if latitude and longitude:
                    latitude = convert_to_degrees(latitude)
                    longitude = convert_to_degrees(longitude)

                    if geotags.get('GPSLatitudeRef') == 'S':
                        latitude = -latitude
                    if geotags.get('GPSLongitudeRef') == 'W':
                        longitude = -longitude

                    return latitude, longitude

    except Exception as e:
        raise ValueError(f"Error while extracting GPS information from {image_path}") from e

    return None

# Replace 'your_image_path' with the actual path to your image.
image_path = '/content/DJI_0302.jpg'
coordinates = get_gps_coordinates(image_path)
if coordinates:
    latitude, longitude = coordinates
    print(f"Image: {image_path}, GPS Latitude: {latitude}, GPS Longitude: {longitude}")

    # Set up Earth Engine
    ee.Initialize()

    # Create a point of interest as an ee.Geometry
    poi = ee.Geometry.Point([longitude, latitude])

    # Get Sentinel-2 image collection
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR") \
        .filterBounds(poi) \
        .filterDate('2022-01-01', '2022-12-31') \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

    # Calculate NDVI for each image in the collection
    def calculate_ndvi(image):
        ndvi = image.normalizedDifference(['B5', 'B4'])
        return image.addBands(ndvi.rename('NDVI'))

    sentinel_with_ndvi = sentinel.map(calculate_ndvi)

    # Select the first image in the collection
    first_image = ee.Image(sentinel_with_ndvi.first())

    # Get the NDVI band
    ndvi_band = first_image.select('NDVI')

    # Print summary information
    print("Summary Information:")
    print(f"NDVI: {ndvi_band.reduceRegion(ee.Reducer.mean(), poi).get('NDVI').getInfo()}")

    # Define water quality parameters using Sentinel-2 bands
    water_quality_params = first_image.select(['B3', 'B4', 'B8A'])

    # Print water quality summary information
    print(f"Water Quality Parameters: {water_quality_params.reduceRegion(ee.Reducer.mean(), poi).getInfo()}")

    # Calculate NDWI
    ndwi = first_image.normalizedDifference(['B3', 'B8A'])
    ndwi_mean = ndwi.reduceRegion(ee.Reducer.mean(), poi).get('nd').getInfo()
    print(f"NDWI: {ndwi_mean}")

    # Determine water quality based on NDWI
    if ndwi_mean < 0.2:
        print("Water quality is good.")
    elif 0.2 <= ndwi_mean < 0.5:
        print("Water quality is moderate.")
    else:
        print("Water quality is poor. Action may be needed.")

    # Display the map
    print("Displaying the map...")
    map_center = [latitude, longitude]
    map_zoom = 15

    # Create a folium map centered at the GPS coordinates
    mymap = folium.Map(location=map_center, zoom_start=map_zoom)

    # Add a marker for the image location
    folium.Marker(location=map_center, popup="Image Location").add_to(mymap)

    # Display the image on the map using folium.ImageOverlay
    image_url = f'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/{first_image.getMapId()}?token={first_image.getMapId()}'
    image_overlay = folium.raster_layers.ImageOverlay(
        image_url,
        bounds=[[latitude, longitude]],
        opacity=0.6,
    ).add_to(mymap)

    # Display the map inline
    display(mymap)
else:
    print(f"Image: {image_path}, GPS information not found or invalid GPS coordinate format.")

Image: /content/DJI_0302.jpg, GPS Latitude: 10.895795, GPS Longitude: 106.69533333333334
Summary Information:
NDVI: 0.04253416783752714
Water Quality Parameters: {'B3': 4046, 'B4': 3940, 'B8A': 4277}
NDWI: -0.027754415475189236
Water quality is good.
Displaying the map...


In [14]:
import requests
import json
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import folium
from folium import plugins
from datetime import datetime as dt

def get_geotagging(exif):
    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging

def convert_to_degrees(value):
    d = value[0]
    m = value[1]
    s = value[2]
    return d + (m / 60.0) + (s / 3600.0)

def get_gps_coordinates(image_path):
    try:
        img = Image.open(image_path)
        exif_data = img._getexif()

        if exif_data is not None:
            geotags = get_geotagging(exif_data)
            if geotags:
                latitude = geotags.get('GPSLatitude')
                longitude = geotags.get('GPSLongitude')

                if latitude and longitude:
                    latitude = convert_to_degrees(latitude)
                    longitude = convert_to_degrees(longitude)

                    if geotags.get('GPSLatitudeRef') == 'S':
                        latitude = -latitude
                    if geotags.get('GPSLongitudeRef') == 'W':
                        longitude = -longitude

                    return latitude, longitude

    except Exception as e:
        raise ValueError(f"Error while extracting GPS information from {image_path}") from e

    return None

# Replace 'your_image_path' with the actual path to your image.
image_path = '/content/DJI_0302.jpg'
coordinates = get_gps_coordinates(image_path)
if coordinates:
    latitude, longitude = coordinates
    print(f"Image: {image_path}, GPS Latitude: {latitude}, GPS Longitude: {longitude}")

    # Set up Earth Engine
    ee.Initialize()

    # Create a point of interest as an ee.Geometry
    poi = ee.Geometry.Point([longitude, latitude])

    # Get Sentinel-2 image collection
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR") \
        .filterBounds(poi) \
        .filterDate('2022-01-01', '2022-12-31') \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

    # Calculate NDVI for each image in the collection
    def calculate_ndvi(image):
        ndvi = image.normalizedDifference(['B5', 'B4'])
        return image.addBands(ndvi.rename('NDVI'))

    sentinel_with_ndvi = sentinel.map(calculate_ndvi)

    # Select the first image in the collection
    first_image = ee.Image(sentinel_with_ndvi.first())

    # Get the NDVI band
    ndvi_band = first_image.select('NDVI')

    # Print summary information
    print("Summary Information:")
    print(f"NDVI: {ndvi_band.reduceRegion(ee.Reducer.mean(), poi).get('NDVI').getInfo()}")

    # Define water quality parameters using Sentinel-2 bands
    water_quality_params = first_image.select(['B3', 'B4', 'B8A'])

    # Print water quality summary information
    print(f"Water Quality Parameters: {water_quality_params.reduceRegion(ee.Reducer.mean(), poi).getInfo()}")

    # Calculate NDWI
    ndwi = first_image.normalizedDifference(['B3', 'B8A'])
    ndwi_mean = ndwi.reduceRegion(ee.Reducer.mean(), poi).get('nd').getInfo()
    print(f"NDWI: {ndwi_mean}")

    # Determine water quality based on NDWI
    if ndwi_mean < 0.2:
        print("Water quality is good.")
    elif 0.2 <= ndwi_mean < 0.5:
        print("Water quality is moderate.")
    else:
        print("Water quality is poor. Action may be needed.")

    # Display the map
    print("Displaying the map...")
    map_center = [latitude, longitude]
    map_zoom = 15
    map_width = 500
    map_height = 500

    # Create a folium map with specified dimensions
    mymap = folium.Map(location=map_center, zoom_start=map_zoom, width=map_width, height=map_height)

    # Add a marker for the image location
    folium.Marker(location=map_center, popup="Image Location").add_to(mymap)

    # Display the image on the map using folium.ImageOverlay
    image_url = f'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/{first_image.getMapId()}?token={first_image.getMapId()}'
    image_overlay = folium.raster_layers.ImageOverlay(
        image_url,
        bounds=[[latitude, longitude]],
        opacity=0.6,
    ).add_to(mymap)

    # Display the map inline
    display(mymap)
else:
    print(f"Image: {image_path}, GPS information not found or invalid GPS coordinate format.")


Image: /content/DJI_0302.jpg, GPS Latitude: 10.895795, GPS Longitude: 106.69533333333334
Summary Information:
NDVI: 0.04253416783752714
Water Quality Parameters: {'B3': 4046, 'B4': 3940, 'B8A': 4277}
NDWI: -0.027754415475189236
Water quality is good.
Displaying the map...
